In [33]:
#自然语言处理：LDA，潜在迪利克雷分布分析、意见挖掘、情感分析。。。。。。
#索引器：一种网页存储方式，将爬虫爬取到的网页存储到结构化数据库。

import os
import numpy
from bs4 import BeautifulSoup              #读取目录下的文件，使用美味唐解析网页内容
moviehtmldir = './data/movie/'
moviedict = {}
for filename in [f for f in os.listdir(moviehtmldir) if f[0]!='.']:
    id = filename.split('.')[0]
    f = open(moviehtmldir+'/'+filename,encoding='unicode_escape')
    parsed_html = BeautifulSoup(f.read())
    try:
        title = parsed_html.body.h1.text
    except:
        title = 'none'
    moviedict[id] = title

In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

tknzr = WordPunctTokenizer()                      #实例一个对象
nltk.download('stopwords')                        #从nltk库中下载停用词
stoplist = stopwords.words('english')             #英文停用词   
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def ListDocs(dirname):                            #对每一个文件解析出来的信息进行处理
    docs = []
    titles = []
    for filename in [f for f in os.listdir(dirname) if str(f)[0]!='.']:#所有不为空的文件
        f = open(dirname+'/'+filename,'r')
        id = filename.split('.')[0].split('_')[1]  #分割
        titles.append(moviedict[id])
        docs.append(f.read())
    return docs,titles

dir = './data/review_polarity/txt_sentoken/'
pos_textreviews,pos_titles = ListDocs(dir+'pos/')
neg_textreviews,neg_titles = ListDocs(dir+'neg/')
tot_textreviews = pos_textreviews+neg_textreviews
tot_titles = pos_titles+neg_titles


[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>


KeyError: '29590'

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

def PreprocessTfidf(texts,stoplist=[],stem=False):  #对文档进行预处理(删除停用词、分词、提取词干)
    newtexts = []
    for text in texts:
        if stem:
           tmp = [w for w in tknzr.tokenize(text) if w not in stoplist] #文档中不在停用词列表中返回
        else:
           tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
        newtexts.append(' '.join(tmp))
    return newtexts
vectorizer = TfidfVectorizer(min_df=1)
processed_reviews = PreprocessTfidf(tot_textreviews,stoplist,True)#调用预处理函数
mod_tfidf = vectorizer.fit(processed_reviews)        #模型拟合
vec_tfidf = mod_tfidf.transform(processed_reviews)
tfidf = dict(list(zip(vectorizer.get_feature_names(),vectorizer.idf_)))


NameError: name 'tot_textreviews' is not defined

In [16]:
import pickle as pickle             #pickle库，实现python的持久化存储 
#print mod_tfidf.get_feature_names()#将得到的结果保存到文件中
print(len(processed_reviews),'--',len(mod_tfidf.get_feature_names()))
v= mod_tfidf.transform(processed_reviews)
with open('./data/vectorizer.pk', 'wb') as fin:
      pickle.dump(mod_tfidf, fin)    #写入文件
file = open("vectorizer.pk",'r')
load_tfidf =  pickle.load(file)
        
print(load_tfidf.transform(PreprocessTfidf([' '.join(['drama'])],stoplist,True)))

NameError: name 'processed_reviews' is not defined

In [27]:
#使用LSA模型
import gensim
from gensim import models
class GenSimCorpus(object):
           def __init__(self, texts, stoplist=[],stem=False):#类的初始化函数
               self.texts = texts
               self.stoplist = stoplist
               self.stem = stem
               self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))
               
            
           def __len__(self):                          #返回文本长度
               return len(self.texts)                  
           def __iter__(self):
               for tokens in self.iter_docs(self.texts, self.stoplist):
                   yield self.dictionary.doc2bow(tokens)
           def iter_docs(self,texts, stoplist):
               for text in texts:
                   if self.stem:
                      yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
                   else:
                      yield (x for x in tknzr.tokenize(text) if x not in stoplist)

corpus = GenSimCorpus(tot_textreviews,stoplist,True)
dict_corpus = corpus.dictionary
ntopics = 10
lsi =  models.LsiModel(corpus, num_topics=ntopics, id2word=dict_corpus)


NameError: name 'tot_textreviews' is not defined

In [28]:
U = lsi.projection.u                     #从模型中返回的Lsi对象得到U、V、S矩阵
Sigma = np.eye(ntopics)*lsi.projection.s   #计算得到矩阵V，调用gensim进行稀疏矩阵操作
V = gensim.matutils.corpus2dense(lsi[corpus], len(lsi.projection.s)).T / lsi.projection.s
dict_words = {}
for i in range(len(dict_corpus)):
    dict_words[dict_corpus[i]] = i

NameError: name 'lsi' is not defined

In [31]:
from collections import namedtuple

def PreprocessDoc2Vec(text,stop=[],stem=False):  #与处理数据，要转化为Doc2Vec能够处理的格式
    words = tknzr.tokenize(text)
    if stem:
       words_clean = [stemmer.stem(w) for w in [i.lower() for i in words if i not in stop]]
    else:                                        #同样的主要判断是否在停用词中进行处理
       words_clean = [i.lower() for i in words if i not in stop]
    return words_clean

Review = namedtuple('Review','words tags')
dir = './data/review_polarity/txt_sentoken/'
do2vecstem = False
reviews_pos = []
cnt = 0                                         #对于文件夹下每一个文档都进行预处理
for filename in [f for f in os.listdir(dir+'pos/') if str(f)[0]!='.']:
    f = open(dir+'pos/'+filename,'r')
    reviews_pos.append(Review(PreprocessDoc2Vec(f.read(),stoplist,do2vecstem),['pos_'+str(cnt)]))
    cnt+=1
    
reviews_neg = []
cnt= 0
for filename in [f for f in os.listdir(dir+'neg/') if str(f)[0]!='.']:
    f = open(dir+'neg/'+filename,'r')
    reviews_neg.append(Review(PreprocessDoc2Vec(f.read(),stoplist,do2vecstem),['neg_'+str(cnt)]))
    cnt+=1

tot_reviews = reviews_pos + reviews_neg

In [32]:
from gensim.models import Doc2Vec
import multiprocessing

cores = multiprocessing.cpu_count()          #线程数
vec_size = 500
model_d2v = Doc2Vec(dm=1, dm_concat=0, size=vec_size, window=10, negative=0, hs=0, min_count=1, workers=cores)
        #DM架构为1，隐含层为500，窗口大小为10个单词，negative（负采样），hs（层次）
model_d2v.build_vocab(tot_reviews)         #训练模型
numepochs= 20
for epoch in range(numepochs):
    try:
        print('epoch %d' % (epoch))
        model_d2v.train(tot_reviews)
        model_d2v.alpha *= 0.99           #学习速率为0.99
        model_d2v.min_alpha = model_d2v.alpha
    except (KeyboardInterrupt, SystemExit):
        break

TypeError: __init__() got an unexpected keyword argument 'size'

In [35]:
query = ['science','future','action']  #设置检索的查询词
#使用TF-IDF
from sklearn.metrics.pairwise import cosine_similarity
query_vec = mod_tfidf.transform(PreprocessTfidf([' '.join(query)],stoplist,True))
sims= cosine_similarity(query_vec,vec_tfidf)[0]  #cosine_similarity函数将稀疏向量转换为常规向量
#这是所有与查询词相似的网页
indxs_sims = sims.argsort()[::-1]
for d in list(indxs_sims)[:5]:               #取排序之后前五个最相似的网页
    print('sim:',sims[d],' title:',tot_titles[d])

#使用LSA模型处理，方法类似
def TransformWordsListtoQueryVec(wordslist,dict_words,stem=False):
    q = np.zeros(len(list(dict_words.keys())))
    for w in wordslist:
        if stem:
            q[dict_words[stemmer.stem(w)]]=1.   #stemmer.stem(w)词干提取
        else:
            q[dict_words[w]] = 1.
    return q
q = TransformWordsListtoQueryVec(query,dict_words,True)#词列表转换为查询向量
qk =   np.dot(np.dot(q,U),Sigma)
sims = np.zeros(len(tot_textreviews))
for d in range(len(V)):
    sims[d]=np.dot(qk,V[d])
indxs_sims = np.argsort(sims)[::-1]  
for d in list(indxs_sims)[:5]:
    print('sim:',sims[d],' doc:',tot_titles[d])

#doc2vec模型
model_d2v.random = np.random.RandomState(1)    #使用RandomState获得随机数生成器
query_docvec = model_d2v.infer_vector(PreprocessDoc2Vec(' '.join(query),stoplist,do2vecstem))
reviews_related = model_d2v.docvecs.most_similar([query_docvec], topn=5)#model_d2v.docvecs.most_similar([query_docvec], topn=3)
for review in reviews_related:
    print('relevance:',review[1],'  title:',tot_titles[review[0]])

NameError: name 'mod_tfidf' is not defined

In [37]:
###########################潜在迪利克雷分配(LDA)#################################
#一些变量可以用潜在的、未被观察到的变量解释
#LDA主题分类示例

import gensim.models
from gensim import models
from nltk.tokenize import RegexpTokenizer
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()                    #词干提取
class GenSimCorpus(object):
           def __init__(self, texts, stoplist=[],bestwords=[],stem=False):
               self.texts = texts   #初始化函数
               self.stoplist = stoplist
               self.stem = stem
               self.bestwords = bestwords
               self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))
            
           def __len__(self):
               return len(self.texts)
           def __iter__(self):
               for tokens in self.iter_docs(self.texts, self.stoplist):
                   yield self.dictionary.doc2bow(tokens)
           def iter_docs(self,texts, stoplist):
               for text in texts:
                   if self.stem:    
                      yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
                   else:
                      if len(self.bestwords)>0:
                         yield (x for x in tknzr.tokenize(text) if x in self.bestwords)
                      else:
                         yield (x for x in tknzr.tokenize(text) if x not in stoplist)            
            
num_topics = 10                 #设置主题的个数为10
corpus = GenSimCorpus(tot_textreviews, stoplist,[],False)
dict_lda = corpus.dictionary
lda = models.LdaModel(corpus, num_topics=num_topics, id2word=dict_lda,passes=10, iterations=50)
print(lda.show_topics(num_topics=num_topics))

NameError: name 'tot_textreviews' is not defined

In [38]:
import copy
out_ids = [tokenid for tokenid, docfreq in dict_lda.dfs.items() if docfreq > 1000 or docfreq < 3 ]
dict_lfq = copy.deepcopy(dict_lda)  #深度复制，得到新的独立的对象
dict_lfq.filter_tokens(out_ids)     #过滤掉不会给网页增加信息、出现频率高的单词
dict_lfq.compactify()    #过滤掉了id，此函数使字典更紧凑
corpus = [dict_lfq.doc2bow(tknzr.tokenize(text)) for text in tot_textreviews]

lda_lfq = models.LdaModel(corpus, num_topics=num_topics, id2word=dict_lfq,passes=10, iterations=50,alpha=0.01,eta=0.01)
for t in range(num_topics):
    print('topic ',t,'  words: ',lda_lfq.print_topic(t,topn=10))
    print()

def GenerateDistrArrays(corpus):     #查询所有属于主题6的电影
     for i,dist in enumerate(corpus[:10]):
         dist_array = np.zeros(num_topics)
         for d in dist:
             dist_array[d[0]] =d[1]
         if dist_array.argmax() == 6 :
            print(tot_titles[i])
corpus_lda = lda_lfq[corpus]
GenerateDistrArrays(corpus_lda)

NameError: name 'dict_lda' is not defined

In [41]:
#########################观点挖掘（情感分析）####################################
#观点挖掘的标准方法是将情感积极或消极作为分类的目标类别
#

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()
from nltk.tokenize import RegexpTokenizer
from collections import namedtuple
from nltk.stem.porter import PorterStemmer
#collections模块的namedtuple子类不仅可以使用item的index访问item，还可以通过item的name进行访问
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)#正则表达式匹配
nltk.download('stopwords')    #导入停用词（手动下载完成）
stoplist = stopwords.words('english')#语言为英文
stemmer = PorterStemmer()     #初始化对象，用于词干提取

def PreprocessReviews(text,stop=[],stem=False):     #预处理函数，对词干进行提取
    words = tknzr.tokenize(text)
    if stem:
       words_clean = [stemmer.stem(w) for w in [i.lower() for i in words if i not in stop]]
    else:
       words_clean = [i.lower() for i in words if i not in stop]
    return words_clean

Review = namedtuple('Review','words title tags')
dir = './data/review_polarity/txt_sentoken/'
do2vecstem = True
reviews_pos = []
cnt = 0
for filename in [f for f in os.listdir(dir+'pos/') if str(f)[0]!='.']: 
    f = open(dir+'pos/'+filename,'r')
    id = filename.split('.')[0].split('_')[1]
    reviews_pos.append(Review(PreprocessReviews(f.read(),stoplist,do2vecstem),moviedict[id],['pos_'+str(cnt)]))
    cnt+=1
    
reviews_neg = []
cnt= 0
for filename in [f for f in os.listdir(dir+'neg/') if str(f)[0]!='.']:
    f = open(dir+'neg/'+filename,'r')
    id = filename.split('.')[0].split('_')[1]
    reviews_neg.append(Review(PreprocessReviews(f.read(),stoplist,do2vecstem),moviedict[id],['neg_'+str(cnt)]))
    cnt+=1

tot_reviews = reviews_pos + reviews_neg
#切分数据及，训练集（80%）、测试机（20%），并且转换数据格式，使其能够被nltk库处理
def word_features(words):
    return dict([(word, True) for word in words])
negfeatures = [(word_features(r.words), 'neg') for r in reviews_neg]#负向特征
posfeatures = [(word_features(r.words), 'pos') for r in reviews_pos]#正向特征
portionpos = int(len(posfeatures)*0.8)
portionneg = int(len(negfeatures)*0.8)
print(portionpos,'-',portionneg)
trainfeatures = negfeatures[:portionneg] + posfeatures[:portionpos]#训练集
print(len(trainfeatures))
testfeatures = negfeatures[portionneg:] + posfeatures[portionpos:]#测试集
#使用朴素贝叶斯分类器训练和测试分类器
from nltk.classify import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(trainfeatures) #初始化分类器对象
err = 0
print('test on: ',len(testfeatures))
for r in testfeatures:         #使用朴素贝叶斯对测试集进行分类
    sent = classifier.classify(r[0])
    if sent != r[1]:
       err +=1.
print('error rate: ',err/float(len(testfeatures)))

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     远程主机强迫关闭了一个现有的连接。>


KeyError: '29590'

In [42]:
#使用卡方方法选择最佳二元组特征
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from random import shuffle
#用于查找和排列bigram搭配或其他关联度量的工具
def bigrams_words_features(words, nbigrams=200,measure=BigramAssocMeasures.chi_sq):
    bigram_finder = BigramCollocationFinder.from_words(words)
    #把词列表变为双词搭配。为给定序列中的所有bigrams构建一个BigramCollocationFinder。
    bigrams = bigram_finder.nbest(measure, nbigrams)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])

negfeatures = [(bigrams_words_features(r.words,500), 'neg') for r in reviews_neg]
posfeatures = [(bigrams_words_features(r.words,500), 'pos') for r in reviews_pos]
portionpos = int(len(posfeatures)*0.8)
portionneg = int(len(negfeatures)*0.8)
print(portionpos,'-',portionneg)
trainfeatures = negfeatures[:portionpos] + posfeatures[:portionneg]
print(len(trainfeatures))
classifier = NaiveBayesClassifier.train(trainfeatures)#使用朴素贝叶斯分类器分类

testfeatures = negfeatures[portionneg:] + posfeatures[portionpos:]
shuffle(testfeatures)                #打乱原测试特征的顺序
err = 0
print('test on: ',len(testfeatures))
for r in testfeatures:
    sent = classifier.classify(r[0])
    if sent != r[1]:
       err +=1.
print('error rate: ',err/float(len(testfeatures)))

0 - 800
0


NameError: name 'NaiveBayesClassifier' is not defined

In [43]:
import nltk.classify.util, nltk.metrics
tot_poswords = [val for l in [r.words for r in reviews_pos] for val in l]
tot_negwords = [val for l in [r.words for r in reviews_neg] for val in l]
from nltk.probability import FreqDist, ConditionalFreqDist
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()
#计算每个单词在全部语料、积极语料、消极语料词频，抽取前一万
for word in tot_poswords:
    word_fd[word.lower()] +=1
    label_word_fd['pos'][word.lower()] +=1
for word in tot_negwords:
    word_fd[word.lower()] +=1
    label_word_fd['neg'][word.lower()] +=1
pos_words = len(tot_poswords)
neg_words = len(tot_negwords)
tot_words = pos_words + neg_words
word_scores = {}
 
for word, freq in word_fd.items():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
                (freq, pos_words), tot_words)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
                (freq, neg_words), tot_words)
    word_scores[word] = pos_score + neg_score
print('total: ',len(word_scores))
best = sorted(iter(word_scores.items()), key=lambda w_s: w_s[1], reverse=True)[:10000]
bestwords = set([w for w, s in best])

def best_words_features(words):
    return dict([(word, True) for word in words if word in bestwords])

negfeatures = [(best_words_features(r.words), 'neg') for r in reviews_neg]
posfeatures = [(best_words_features(r.words), 'pos') for r in reviews_pos]
portionpos = int(len(posfeatures)*0.8)
portionneg = int(len(negfeatures)*0.8)
print(portionpos,'-',portionneg)
trainfeatures = negfeatures[:portionpos] + posfeatures[:portionneg]
print(len(trainfeatures))
classifier = NaiveBayesClassifier.train(trainfeatures)
##test with feature chi square selection
testfeatures = negfeatures[portionneg:] + posfeatures[portionpos:]
shuffle(testfeatures)
err = 0
print('test on: ',len(testfeatures))
for r in testfeatures:
    sent = classifier.classify(r[0])
    #print r[1],'-pred: ',sent
    if sent != r[1]:
       err +=1.
print('error rate: ',err/float(len(testfeatures)))

ZeroDivisionError: division by zero